In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 33.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import emoji
import unicodedata

# ==============================================================================
# 1. CẤU HÌNH INPUT (Điền tất cả đường dẫn file vào đây)
# ==============================================================================
ALL_FILES = [
    # --- Nhóm 3 file cũ (Social) ---
    "/content/train.csv",
    "/content/val.csv",
    "/content/test.csv",

    # --- File mới (News) ---
    "/content/vn_news_226_tlfr.csv"
]

OUTPUT_FILE = "FINAL_DATASET_MERGED.csv"

# ==============================================================================
# 2. HÀM CLEANING (CHUẨN HÓA & GIỮ NGÀY THÁNG)
# ==============================================================================
def smart_clean(text):
    if not isinstance(text, str): return ""

    # 1. Fix lỗi font (Unicode NFC) & Xóa xuống dòng
    text = unicodedata.normalize('NFC', text)
    text = text.replace("_", " ").replace("\n", " . ")

    # 2. Xóa Emoji & URL
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r'(https?://\S+|www\.\S+|<URL>)', '', text)

    # 3. Cắt rác cuối câu (Xem thêm, Nguồn...)
    text = re.sub(r'(Xem thêm.*|Theo .*|Nguồn:.*|Ảnh:.*|Cre:.*)$', '', text, flags=re.I|re.M)

    # 4. GIỮ: Chữ, Số, Dấu câu cơ bản, Dấu / (ngày tháng)
    text = re.sub(r'[^\w\s.,!?;:\%\-\(\)\/]', ' ', text)

    # 5. Fix lỗi khoảng trắng (10 / 4 -> 10/4, . . -> .)
    text = re.sub(r'\s*/\s*', '/', text)
    text = re.sub(r'\.\s*\.', '.', text)

    return re.sub(r'\s+', ' ', text).strip()

# ==============================================================================
# 3. PIPELINE GỘP & XỬ LÝ
# ==============================================================================
try:
    print("🚀 Đang xử lý và gộp dữ liệu...")
    dfs = []

    for f in ALL_FILES:
        # Đọc file (chưa lọc cột vội để check tên cột)
        try:
            temp_df = pd.read_csv(f, dtype=str) # Đọc tất cả là string để tránh lỗi

            # --- TỰ ĐỘNG CHUẨN HÓA TÊN CỘT ---
            # Nếu thấy 'post_message' thì đổi thành 'text'
            if 'post_message' in temp_df.columns:
                temp_df.rename(columns={'post_message': 'text'}, inplace=True)

            # Chỉ giữ lại 'text' và 'label', bỏ qua nếu file thiếu cột
            if 'text' in temp_df.columns and 'label' in temp_df.columns:
                dfs.append(temp_df[['text', 'label']])
            else:
                print(f"⚠️ Bỏ qua file {f}: Không tìm thấy cột 'text' hoặc 'label'")

        except Exception as e:
            print(f"❌ Lỗi đọc file {f}: {e}")

    # Gộp tất cả
    if not dfs:
        raise ValueError("Không có dữ liệu nào được load!")

    df = pd.concat(dfs, ignore_index=True)
    print(f"📊 Tổng số dòng thô: {len(df)}")

    # --- CLEANING & FILTERING ---
    print("🧹 Đang làm sạch text...")
    df['text'] = df['text'].apply(smart_clean)

    # Xử lý cột Label (về 0 và 1)
    df['label'] = pd.to_numeric(df['label'], errors='coerce').fillna(-1).astype(int)
    df = df[df['label'].isin([0, 1])] # Chỉ lấy label 0 hoặc 1

    # Lọc text rác (ngắn < 15 ký tự)
    df = df[df['text'].str.len() > 15]

    # Xóa trùng lặp
    before = len(df)
    df.drop_duplicates(subset=['text'], keep='first', inplace=True)
    print(f"✨ Đã xóa {before - len(df)} dòng trùng lặp.")

    # --- LƯU FILE ---
    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("\n" + "="*40)
    print(f"✅ XONG! File cuối cùng: {OUTPUT_FILE}")
    print(f"👉 Số lượng mẫu sạch: {len(df)}")
    print(f"👉 Mẫu dữ liệu:")
    print(df.head(3))
    print("="*40)

except Exception as e:
    print(f"❌ CRITICAL ERROR: {e}")

🚀 Đang xử lý và gộp dữ liệu...
📊 Tổng số dòng thô: 9939
🧹 Đang làm sạch text...
✨ Đã xóa 270 dòng trùng lặp.

✅ XONG! File cuối cùng: FINAL_DATASET_MERGED.csv
👉 Số lượng mẫu sạch: 9375
👉 Mẫu dữ liệu:
                                                text  label
0  ĐẾN 18H CHIỀU 10/4 : THÊM 2 CA MẮC COVID-19 BN...      0
1  Thế giới tuần qua: Nợ công Mỹ tăng kinh hoàng ...      0
2  Sự cố đứt cáp AAG không ảnh hưởng tới chất lượ...      0


In [ ]:
OUTPUT_FILE = "DATASET_FOR_EDA_RAW.csv"

# ==============================================================================
# 2. HÀM CLEAN NHẸ (MINIMAL CLEAN)
# ==============================================================================
def raw_clean(text):
    """
    Chỉ sửa lỗi kỹ thuật, giữ nguyên nội dung gốc (Emoji, Teencode...)
    """
    if not isinstance(text, str): return ""

    # 1. Fix lỗi vỡ font tiếng Việt (Bắt buộc phải làm để máy đọc được chữ)
    # Ví dụ: "đu ng" (NFD) -> "đúng" (NFC)
    text = unicodedata.normalize('NFC', text)

    # 2. Xóa dấu gạch dưới "_" do bộ tách từ cũ tạo ra
    # Cái này là nhiễu kỹ thuật, không phải do người dùng viết, nên xóa cho dễ đọc
    # "quốc_tịch" -> "quốc tịch"
    text = text.replace("_", " ")

    # 3. Xử lý xuống dòng để CSV không bị lỗi
    # (Tùy chọn: Nếu muốn giữ nguyên xuống dòng trong ô Excel thì bỏ dòng này)
    # Nhưng để code Python đọc dễ thì nên thay bằng khoảng trắng hoặc dấu chấm
    # text = text.replace("\n", " ")

    return text.strip()

# ==============================================================================
# 3. PIPELINE GỘP DỮ LIỆU
# ==============================================================================
try:
    print("🚀 Đang gộp dữ liệu để làm EDA...")
    dfs = []

    for f in ALL_FILES:
        try:
            # Đọc file, ép kiểu string để không lỗi
            df_temp = pd.read_csv(f, dtype=str)

            # Đổi tên cột post_message thành text nếu có
            if 'post_message' in df_temp.columns:
                df_temp.rename(columns={'post_message': 'text'}, inplace=True)

            # Chỉ lấy 2 cột quan trọng
            if 'text' in df_temp.columns and 'label' in df_temp.columns:
                dfs.append(df_temp[['text', 'label']])
            else:
                print(f"⚠️ File {f} thiếu cột text/label -> Bỏ qua.")

        except Exception as e:
            print(f"❌ Lỗi đọc file {f}: {e}")

    # Gộp lại
    df = pd.concat(dfs, ignore_index=True)
    print(f"📊 Tổng số dòng thu thập: {len(df)}")

    # --- XỬ LÝ NHẸ ---
    # 1. Chạy hàm clean nhẹ
    df['text'] = df['text'].apply(raw_clean)

    # 2. Xử lý Label về dạng số (0 và 1)
    # Ép kiểu sang số, cái nào lỗi thành NaN rồi xóa
    df['label'] = pd.to_numeric(df['label'], errors='coerce')
    df = df.dropna(subset=['label']) # Xóa dòng không có nhãn
    df['label'] = df['label'].astype(int)

    # 3. Xóa các dòng text quá ngắn hoặc rỗng (NULL)
    # Để EDA chính xác thì ít nhất phải có chữ
    df = df.dropna(subset=['text'])
    df = df[df['text'].str.strip().str.len() > 0]

    # 4. Xóa trùng lặp (Tuyệt đối trùng nhau)
    # Vẫn nên xóa trùng lặp hoàn toàn để biểu đồ không bị bias
    before = len(df)
    df.drop_duplicates(subset=['text'], keep='first', inplace=True)
    print(f"✨ Đã xóa {before - len(df)} bài trùng lặp hoàn toàn.")

    # --- LƯU FILE ---
    # utf-8-sig giúp mở bằng Excel trên Windows không bị lỗi font tiếng Việt
    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("\n" + "="*40)
    print(f"✅ ĐÃ XUẤT FILE EDA: {OUTPUT_FILE}")
    print(f"👉 Số lượng mẫu: {len(df)}")
    print(f"👉 Dữ liệu mẫu (Vẫn còn Emoji):")
    print(df[['text', 'label']].head(3))
    print("="*40)

except Exception as e:
    print(f"❌ Lỗi: {e}")

🚀 Đang gộp dữ liệu để làm EDA...
📊 Tổng số dòng thu thập: 9939
✨ Đã xóa 103 bài trùng lặp hoàn toàn.

✅ ĐÃ XUẤT FILE EDA: DATASET_FOR_EDA_RAW.csv
👉 Số lượng mẫu: 9836
👉 Dữ liệu mẫu (Vẫn còn Emoji):
                                                text  label
0  ĐẾN 18H CHIỀU 10/4 : THÊM 2 CA MẮC COVID-19 ➡️...      0
1  Thế giới tuần qua: Nợ công Mỹ tăng kinh hoàng ...      0
2  Sự cố đứt cáp AAG không ảnh hưởng tới chất lượ...      0


In [ ]:
import pandas as pd
import numpy as np
import os

# ==============================================================================
# 1. CẤU HÌNH INPUT
# ==============================================================================
FILE_DS1 = "/content/FINAL_DATASET_MERGED.csv"          # File News/Social
FILE_DS2 = "/content/dataset_TIKTOK_FAKE_NEWS_FINAL.csv" # File TikTok
OUTPUT_FILE = "MERGED_DATASET_FINAL.csv"

# ==============================================================================
# 2. HÀM XỬ LÝ (CÓ DEBUG)
# ==============================================================================

def process_dataset1(file_path):
    print(f"\n🔹 Đang xử lý Dataset 1: {file_path}")
    if not os.path.exists(file_path):
        print(f"❌ Lỗi: Không tìm thấy file {file_path}. Hãy upload lại!")
        return pd.DataFrame()

    try:
        df = pd.read_csv(file_path, dtype=str)
        print(f"   -> Đọc được: {len(df)} dòng ban đầu.")
        print(f"   -> Các cột hiện có: {list(df.columns)}")

        # Tự động tìm cột nội dung (text hoặc post_message hoặc content)
        col_map = {}
        if 'text' in df.columns: col_map['text'] = 'content'
        elif 'post_message' in df.columns: col_map['post_message'] = 'content'
        elif 'content' in df.columns: pass # Đã đúng tên
        else:
            print("❌ Lỗi DS1: Không tìm thấy cột nội dung (text/post_message)")
            return pd.DataFrame()

        df.rename(columns=col_map, inplace=True)

        # Tạo title rỗng nếu chưa có
        if 'title' not in df.columns:
            df['title'] = ""

        # Xử lý label
        if 'label' not in df.columns:
            print("⚠️ Cảnh báo DS1: Không có cột label, mặc định gán = 0")
            df['label'] = 0

        return df[['title', 'content', 'label']]

    except Exception as e:
        print(f"❌ Lỗi xử lý DS1: {e}")
        return pd.DataFrame()

def process_dataset2(file_path):
    print(f"\n🔹 Đang xử lý Dataset 2: {file_path}")
    if not os.path.exists(file_path):
        print(f"❌ Lỗi: Không tìm thấy file {file_path}. Hãy upload lại!")
        return pd.DataFrame()

    try:
        df = pd.read_csv(file_path, dtype=str)
        print(f"   -> Đọc được: {len(df)} dòng ban đầu.")
        print(f"   -> Các cột hiện có: {list(df.columns)}")

        # Mapping TikTok
        rename_map = {}
        if 'desc' in df.columns: rename_map['desc'] = 'title'
        if 'text' in df.columns: rename_map['text'] = 'content'

        df.rename(columns=rename_map, inplace=True)

        # Kiểm tra cột bắt buộc
        if 'content' not in df.columns:
             print("❌ Lỗi DS2: Không tìm thấy cột 'text' (đã rename thành content)")
             return pd.DataFrame()

        # Map Label
        if 'category' in df.columns:
            df['category'] = df['category'].str.lower().str.strip()
            label_map = {'fake': 1, 'real': 0, 'tin giả': 1, 'tin thật': 0}
            df['label'] = df['category'].map(label_map)
            # Drop NaN label
            before_drop = len(df)
            df = df.dropna(subset=['label'])
            print(f"   -> Đã loại bỏ {before_drop - len(df)} dòng không xác định được nhãn (fake/real).")
        elif 'label' in df.columns:
            print("   -> Đã có sẵn cột label.")
        else:
            print("❌ Lỗi DS2: Không tìm thấy cột 'category' hoặc 'label'")
            return pd.DataFrame()

        df['title'] = df['title'].fillna("")
        return df[['title', 'content', 'label']]

    except Exception as e:
        print(f"❌ Lỗi xử lý DS2: {e}")
        return pd.DataFrame()

# ==============================================================================
# 3. MAIN (CHẠY TRỰC TIẾP)
# ==============================================================================

print("🚀 BẮT ĐẦU QUÁ TRÌNH GỘP...")

# 1. Xử lý từng file
df1 = process_dataset1(FILE_DS1)
df2 = process_dataset2(FILE_DS2)

print(f"\n📊 Tổng kết trước khi gộp:")
print(f"   - Dataset 1 sạch: {len(df1)} dòng")
print(f"   - Dataset 2 sạch: {len(df2)} dòng")

# 2. Gộp
if len(df1) == 0 and len(df2) == 0:
    print("\n❌ CẢ 2 FILE ĐỀU RỖNG HOẶC LỖI. DỪNG CHƯƠNG TRÌNH.")
else:
    merged_df = pd.concat([df1, df2], ignore_index=True)

    # 3. Format chuẩn
    merged_df['label'] = pd.to_numeric(merged_df['label'], errors='coerce').fillna(0).astype(int)
    merged_df['content'] = merged_df['content'].fillna("")
    merged_df['title'] = merged_df['title'].fillna("")

    # 4. Xóa trùng lặp
    before = len(merged_df)
    merged_df.drop_duplicates(subset=['title', 'content'], keep='first', inplace=True)
    print(f"\n✨ Đã xóa {before - len(merged_df)} dòng trùng lặp khi gộp.")

    # 5. Lưu
    merged_df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("\n" + "="*50)
    print(f"✅ HOÀN TẤT! FILE RA: {OUTPUT_FILE}")
    print(f"📊 TỔNG SỐ MẪU CUỐI CÙNG: {len(merged_df)}")
    print("="*50)
    print(merged_df.head())

🚀 BẮT ĐẦU QUÁ TRÌNH GỘP...

🔹 Đang xử lý Dataset 1: /content/FINAL_DATASET_MERGED.csv
   -> Đọc được: 9375 dòng ban đầu.
   -> Các cột hiện có: ['text', 'label']

🔹 Đang xử lý Dataset 2: /content/dataset_TIKTOK_FAKE_NEWS_FINAL.csv
   -> Đọc được: 2484 dòng ban đầu.
   -> Các cột hiện có: ['url', 'text', 'category', 'desc', 'keyword', 'createTime', 'shareCount', 'commentCount', 'playCount', 'diggCount', 'collectCount', 'author_nickname', 'author_unique_id', 'author_id', 'video_id', 'thumnail_url']
   -> Đã loại bỏ 0 dòng không xác định được nhãn (fake/real).

📊 Tổng kết trước khi gộp:
   - Dataset 1 sạch: 9375 dòng
   - Dataset 2 sạch: 2484 dòng

✨ Đã xóa 0 dòng trùng lặp khi gộp.

✅ HOÀN TẤT! FILE RA: MERGED_DATASET_FINAL.csv
📊 TỔNG SỐ MẪU CUỐI CÙNG: 11859
  title                                            content  label
0        ĐẾN 18H CHIỀU 10/4 : THÊM 2 CA MẮC COVID-19 BN...      0
1        Thế giới tuần qua: Nợ công Mỹ tăng kinh hoàng ...      0
2        Sự cố đứt cáp AAG không ản